In [1]:
import os
import numpy as np
import pyvista as pv


In [2]:
sourceMeshPath = "assets/library/smile_arch_origin.stl"
targetMeshPath = "results/partial_smile_arch.stl"

In [3]:
pv.set_jupyter_backend('trame')  
plotter = pv.Plotter(notebook=True)

source_mesh = pv.read(sourceMeshPath)
target_mesh = pv.read(targetMeshPath)


# Add source mesh in blue
plotter.add_mesh(source_mesh, color='lightblue', opacity=0.7, label='Source Mesh')

# Add target mesh in green
plotter.add_mesh(target_mesh, color='lightgreen', opacity=0.7, label='Target Mesh')

# Add legend
plotter.add_legend()

# Add a title
plotter.add_title('Source and Target Mesh Comparison')

# Show the plot3
plotter.show()

ImportError: Please install `trame` and `ipywidgets` to use this feature.